In [1]:
import numpy as np
import pandas as pd

import os
import json
from pathlib import Path
import gc
import inspect
import sys

import matplotlib.pyplot as plt
from matplotlib import colors
import numpy as np
from pathlib import Path

from copy import deepcopy

data_path = Path('../data')
train_path = data_path / 'arc-agi_training_challenges.json'
train_sols_path = data_path / 'arc-agi_training_solutions.json'
eval_path = data_path / 'arc-agi_evaluation_challenges.json'
eval_sols_path = data_path / 'arc-agi_evaluation_solutions.json'

In [2]:
sys.path.insert(0, '..')
sys.path.insert(0, '../DSL')

from visualization.visualization_utils import *
import solvers

In [3]:
with open(train_path, 'r') as f:
    train_tasks = json.load(f)
with open(train_sols_path, 'r') as f:
    train_sols = json.load(f)
with open(eval_path, 'r') as f:
    eval_tasks = json.load(f)
with open(eval_sols_path, 'r') as f:
    eval_sols = json.load(f)

train_task_labels = sorted(train_tasks.keys())

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from trl import SFTTrainer, SFTConfig
from datasets import Dataset
import torch

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using {DEVICE}")

Using cuda


# Loading model

In [5]:
codegen_tokenizer = AutoTokenizer.from_pretrained("Salesforce/codegen-350M-mono")
codegen = AutoModelForCausalLM.from_pretrained("Salesforce/codegen-350M-mono")
codegen = codegen.to(DEVICE)

/home/amzi/installs/miniforge3/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


# Training data

In [6]:
def grid_to_string(grid):
    return str(grid).replace(" ", "")

In [7]:
def create_training_example(task_label):
    # TODO: More robust sampling of in/out grids
    in_grid = train_tasks[task_label]['train'][0]['input']
    out_grid = train_tasks[task_label]['train'][0]['output']
    solver_def = inspect.getsource(getattr(solvers, f'solve_{task_label}'))
    prompt = solver_def.split("\n", 1)
    prompt = [prompt[0], f'''     """
    Example input: {grid_to_string(in_grid)}
    Example output: {grid_to_string(out_grid)}
    """''', prompt[1]]
    return "\n".join(prompt)

In [8]:
l = train_task_labels[0]
print(create_training_example(l))

def solve_007bbfb7(I):
     """
    Example input: [[0,7,7],[7,7,7],[0,7,7]]
    Example output: [[0,0,0,0,7,7,0,7,7],[0,0,0,7,7,7,7,7,7],[0,0,0,0,7,7,0,7,7],[0,7,7,0,7,7,0,7,7],[7,7,7,7,7,7,7,7,7],[0,7,7,0,7,7,0,7,7],[0,0,0,0,7,7,0,7,7],[0,0,0,7,7,7,7,7,7],[0,0,0,0,7,7,0,7,7]]
    """
    x1 = hupscale(I, THREE)
    x2 = vupscale(x1, THREE)
    x3 = hconcat(I, I)
    x4 = hconcat(x3, I)
    x5 = vconcat(x4, x4)
    x6 = vconcat(x5, x4)
    O = cellwise(x2, x6, ZERO)
    return O



In [9]:
training_examples = [create_training_example(l) for l in train_task_labels] 
codegen_tokenizer.pad_token = codegen_tokenizer.eos_token
dataset = Dataset.from_dict({'text': training_examples})

# Training

In [10]:
training_args = TrainingArguments(
    output_dir='./results',             # Output directory
    overwrite_output_dir=True,          # Overwrite the content of the output directory
    num_train_epochs=1,                 # Number of training epochs
    per_device_train_batch_size=1,      # Batch size per device
    save_steps=10_000,                  # Save checkpoint every 10,000 steps
    save_total_limit=2,                 # Only last 2 models are saved
    logging_steps=500,                  # Log every 500 steps
)

In [11]:
training_args = SFTConfig(
    output_dir='./results',             # Output directory
    overwrite_output_dir=True,          # Overwrite the content of the output directory
    num_train_epochs=1,                 # Number of training epochs
    per_device_train_batch_size=1,      # Batch size per device
    dataset_text_field="text",
    max_seq_length=2048,
    eval_accumulation_steps = 1,
    fp16=True,
)

In [10]:
trainer = SFTTrainer(
    model = codegen,
    tokenizer = codegen_tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = 1536,
    args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 4,
        num_train_epochs = 5,
        learning_rate = 2e-4,
        fp16 = True,
        bf16 = False,
        logging_steps = 1,
        optim = "adamw_torch",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

/home/amzi/installs/miniforge3/lib/python3.12/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/home/amzi/installs/miniforge3/lib/python3.12/site-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/home/amzi/installs/miniforge3/lib/python3.12/site-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/400 [00:00<?, ? examples/s]

In [11]:
trainer.train()

Step,Training Loss
1,0.807400
2,1.026900
3,1.525700
4,1.248300
5,1.768500
6,0.836000
7,0.744400
8,0.778200
9,0.668800
10,0.580700


TrainOutput(global_step=500, training_loss=0.32996345865726473, metrics={'train_runtime': 194.6973, 'train_samples_per_second': 10.272, 'train_steps_per_second': 2.568, 'total_flos': 2368309617623040.0, 'train_loss': 0.32996345865726473, 'epoch': 5.0})

# Scratch work

In [18]:
try:
    del trainer
except:
    pass
gc.collect()
torch.cuda.empty_cache()

In [60]:
def get_output(model, tokenizer, input_string):
    inputs = tokenizer(input_string, return_tensors = 'pt')
    for key in inputs:
        inputs[key] = inputs[key].to(DEVICE)
    outputs = model.generate(**inputs,
                             max_length = 512,
                             do_sample=True,
                             top_k=0,
                             temperature=1.5,)
    text = tokenizer.batch_decode(outputs)[0]
    return text

In [61]:
input1 = """def solve_a9ut373e(I):\n"""
input2 = """def solve_iu6gn8z9(I):\n"""
t1 = get_output(codegen, codegen_tokenizer, input1)
t2 = get_output(codegen, codegen_tokenizer, input2)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [62]:
print(t1)

def solve_a9ut373e(I):
     """
    Example input: [[2,0,5],[2,0,5],[2,0,5],[5,0,2],[0,0,5]]
    Example output: [[2,0,5],[2,0,5],[2,0,5],[5,0,2],[0,0,5]]
    """
    x1 = width(I)
    x2 = palette(I)
    x3 = halve(x1)
    x4 = vupscale(I, x3)
    x5 = rbind(shoot, UNITY)
    x6 = other(x2, ZERO)
    x7 = ofcolor(x4, x6)
    x8 = mapply(x5, x7)
    O = fill(x4, x6, FOUR)
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
 

In [64]:
print(t2)

def solve_iu6gn8z9(I):
     """
    Example input: [[4,0,0],[1,4,0],[2,4,0],[0,0,2],[0,0,4],[4,1,1],[8,8,8]]
    Example output: [[4,3,8],[4,0,8],[8,8,8]]
    """
    x1 = rot90(I)
    x2 = rot180(I)
    x3 = rot270(I)
    x4 = hconcat(I, x1)
    x5 = hconcat(x3, x2)
    O = vconcat(x4, x5)
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    re

In [65]:
eval_task_labels = sorted(eval_tasks.keys())
l = eval_task_labels[0]
I = grid_to_string(eval_tasks[l]['train'][0]['input'])
O = grid_to_string(eval_tasks[l]['train'][0]['output'])

In [66]:
I

'[[8,6],[6,4]]'

In [67]:
O

'[[8,6,8,6,8,6],[6,4,6,4,6,4],[6,8,6,8,6,8],[4,6,4,6,4,6],[8,6,8,6,8,6],[6,4,6,4,6,4]]'

In [68]:
input3 = f'''def solve_a9ut373e(I):
    Example input: {I}
    Example output: {O}
    """'''
input4 = f'''def solve_iu6gn8z9(I):
    Example input: {I}
    Example output: {O}
    """'''
t3 = get_output(codegen, codegen_tokenizer, input3)
t4 = get_output(codegen, codegen_tokenizer, input4)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [69]:
print(t3)

def solve_a9ut373e(I):
    Example input: [[8,6],[6,4]]
    Example output: [[8,6,8,6,8,6],[6,4,6,4,6,4],[6,8,6,8,6,8],[4,6,4,6,4,6],[8,6,8,6,8,6],[6,4,6,4,6,4]]
    """
    O = hupscale(I, THREE)
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    O = vconcat(I, I)
    x1 = hmirror(I)
    x2 = vconcat(x1, I)
    x3 = vmirror(x2)
    x4 = hconcat(x2, x3)
    x5 = hconcat(x4, x2)
    O = hconcat(x5, x4)
    return O
    return O
    return O
    return O
    return O
    O = vconcat(x8, x6)
    x8 = vconcat(x7, x6)
    O = vconcat(x8, x

In [70]:
print(t4)

def solve_iu6gn8z9(I):
    Example input: [[8,6],[6,4]]
    Example output: [[8,6,8,6,8,6],[6,4,6,4,6,4],[6,8,6,8,6,8],[4,6,4,6,4,6],[8,6,8,6,8,6],[6,4,6,4,6,4]]
    """
    O = vmirror(I)
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    x1 = shape(I)
    x2 = objects(I, F, F, T)
    x3 = argmax(x2, size)
    x4 = canvas(x1, THR

In [71]:
I = grid_to_string(eval_tasks[l]['train'][1]['input'])
O = grid_to_string(eval_tasks[l]['train'][1]['output'])
input5 = f'''def solve_a9ut373e(I):
    Example input: {I}
    Example output: {O}
    """'''
input6 = f'''def solve_iu6gn8z9(I):
    Example input: {I}
    Example output: {O}
    """'''
t5 = get_output(codegen, codegen_tokenizer, input5)
t6 = get_output(codegen, codegen_tokenizer, input6)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [72]:
I

'[[7,9],[4,3]]'

In [73]:
O

'[[7,9,7,9,7,9],[4,3,4,3,4,3],[9,7,9,7,9,7],[3,4,3,4,3,4],[7,9,7,9,7,9],[4,3,4,3,4,3]]'

In [74]:
print(t5)

def solve_a9ut373e(I):
    Example input: [[7,9],[4,3]]
    Example output: [[7,9,7,9,7,9],[4,3,4,3,4,3],[9,7,9,7,9,7],[3,4,3,4,3,4],[7,9,7,9,7,9],[4,3,4,3,4,3]]
    """
    O = hsplit(I, THREE)
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O
    return O

In [75]:
print(t6)

def solve_iu6gn8z9(I):
    Example input: [[7,9],[4,3]]
    Example output: [[7,9,7,9,7,9],[4,3,4,3,4,3],[9,7,9,7,9,7],[3,4,3,4,3,4],[7,9,7,9,7,9],[4,3,4,3,4,3]]
    """
    x1 = vmirror(I)
    O = hconcat(I, x1)
    return O
    return O
    return O
    return O
    O = hmirror(x2)
    x2 = vconcat(x2, xO)
    O = vconcat(x3, x2)
    x4 = vmirror(x5)
    x2 = vconcat(x4, x3)
    O = vconcat(x6, x2)
    return O
    return O
    O = cellwise(x3,pattern, TEN)
    return O
    return O
    x7 = astuple(NINE, NINE)
    O = x7(x)
    x8 = canvas(ZERO, x3)
    O = hconcat(x8, x7)
    O = objects(x7, F, F, T)
    return O
    x9 = hconcat(x9, x7)
    O = cmirror(x9)
    O = vconcat(x10, x9)
    O = vconcat(x11, x9)
    x4 = vconcat(x12, x10)
    O = vconcat(x13, x4)
    return O
    O = cellwise(x3, x9, ZERO)
    return O
    O = cellwise(x4, argmin, width)
    return O
    math = canvas(ZERO, x3)
    O = paint(x5, x29)
    O = paint(x11, x2)
    O = paint(x11, x4)
    O = paint(x9, x


In [43]:
dir(codegen)

['T_destination',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_assisted_decoding',
 '_auto_class',
 '_autoset_attn_implementation',
 '_backward_compatibility_gradient_checkpointing',
 '_backward_hooks',
 '_backward_pre_hooks',
 '_beam_search',
 '_buffers',
 '_call_impl',
 '_check_and_enable_flash_attn_2',
 '_check_and_enable_sdpa',
 '_compiled_call_impl',
 '_constrained_beam_search',
 '_contrastive_search',
 '_convert_head_mask_to_5d',
 '_copy_lm_head_original_to_resized',
 '_create_repo',
 '_dispatch_accelerate_model',
 '_dola_decoding',
 '_expand_inputs_for_generation',

In [10]:
from random import randint

In [36]:
G0 = tuple(tuple(0 for i in range(30)) for j in range(30))
G1 = list(list(0 for i in range(30)) for j in range(30))
G2 = tuple(tuple(randint(0,9) for i in range(30)) for j in range(30))
G3 = list(list(randint(0,9) for i in range(30)) for j in range(30))

In [37]:
print(len(codegen_tokenizer(str(G0))['input_ids']))
print(len(codegen_tokenizer(str(G1))['input_ids']))
print(len(codegen_tokenizer(str(G2))['input_ids']))
print(len(codegen_tokenizer(str(G3))['input_ids']))
print(len(codegen_tokenizer(grid_to_string(G1))['input_ids']))
print(len(codegen_tokenizer(grid_to_string(G3))['input_ids']))

1830
1830
1830
1830
1801
1801


In [38]:
G4 = "\n".join(["".join([str(entry) for entry in row]) for row in G1])
G5 = "\n".join(["".join([str(entry) for entry in row]) for row in G3])

In [39]:
print(len(codegen_tokenizer(G4)['input_ids']))
print(len(codegen_tokenizer(G5)['input_ids']))

119
413


COMPACT GRID ENCODINGS MAKE BIG DIFF! But will they help the model?

In [40]:
def grid_to_string_compact(grid):
    return "\n".join(["".join([str(entry) for entry in row]) for row in grid])

In [47]:
compact_lengths = []
with_commas_lengths = []
for label in train_task_labels:
    N = len(train_tasks[label]['train'])
    for i in range(N):
        G1 = train_tasks[label]['train'][i]['input']
        G2 = train_tasks[label]['train'][i]['output']
        compact_lengths.append(len(codegen_tokenizer(grid_to_string_compact(G1))['input_ids']))
        compact_lengths.append(len(codegen_tokenizer(grid_to_string_compact(G2))['input_ids']))
        with_commas_lengths.append(len(codegen_tokenizer(grid_to_string(G1))['input_ids']))
        with_commas_lengths.append(len(codegen_tokenizer(grid_to_string(G2))['input_ids']))

In [48]:
sorted(with_commas_lengths, reverse = True)

[1801,
 1801,
 1801,
 1801,
 1801,
 1801,
 1801,
 1801,
 1801,
 1801,
 1801,
 1801,
 1801,
 1801,
 1801,
 1801,
 1801,
 1801,
 1801,
 1801,
 1801,
 1801,
 1801,
 1801,
 1683,
 1683,
 1683,
 1683,
 1683,
 1683,
 1683,
 1683,
 1683,
 1683,
 1569,
 1459,
 1459,
 1459,
 1459,
 1459,
 1459,
 1459,
 1353,
 1351,
 1243,
 1243,
 1201,
 1201,
 1201,
 1201,
 1153,
 1153,
 1153,
 1153,
 1153,
 1153,
 1153,
 1153,
 1135,
 1105,
 1101,
 1101,
 1101,
 1059,
 1059,
 1059,
 1059,
 1059,
 1059,
 1059,
 1059,
 1059,
 1059,
 1059,
 1059,
 1059,
 1059,
 1059,
 1059,
 1059,
 1059,
 1059,
 1059,
 1021,
 1013,
 1009,
 1009,
 1009,
 969,
 969,
 967,
 961,
 925,
 925,
 925,
 913,
 883,
 883,
 883,
 883,
 883,
 883,
 883,
 883,
 883,
 883,
 883,
 883,
 883,
 883,
 883,
 883,
 883,
 883,
 883,
 883,
 883,
 883,
 883,
 883,
 883,
 883,
 883,
 883,
 881,
 881,
 841,
 841,
 837,
 801,
 801,
 801,
 801,
 801,
 801,
 801,
 801,
 801,
 801,
 801,
 801,
 801,
 801,
 801,
 801,
 801,
 801,
 801,
 801,
 801,
 801,
 801,


In [49]:
sorted(compact_lengths, reverse = True)

[434,
 413,
 405,
 399,
 389,
 388,
 381,
 379,
 379,
 377,
 376,
 376,
 369,
 368,
 367,
 362,
 357,
 350,
 348,
 334,
 329,
 328,
 328,
 326,
 323,
 319,
 317,
 306,
 302,
 302,
 296,
 296,
 281,
 274,
 271,
 266,
 266,
 263,
 252,
 252,
 251,
 251,
 251,
 246,
 246,
 245,
 244,
 243,
 238,
 234,
 233,
 233,
 233,
 231,
 230,
 225,
 219,
 218,
 216,
 216,
 214,
 212,
 211,
 211,
 211,
 209,
 207,
 204,
 204,
 203,
 202,
 202,
 202,
 201,
 199,
 198,
 196,
 195,
 194,
 193,
 193,
 192,
 191,
 186,
 184,
 183,
 183,
 183,
 181,
 179,
 179,
 178,
 178,
 177,
 177,
 177,
 177,
 175,
 175,
 174,
 174,
 174,
 173,
 171,
 170,
 170,
 169,
 169,
 168,
 166,
 166,
 166,
 161,
 160,
 160,
 160,
 160,
 159,
 158,
 157,
 157,
 155,
 154,
 154,
 154,
 152,
 150,
 150,
 150,
 149,
 149,
 149,
 149,
 149,
 146,
 146,
 145,
 145,
 145,
 143,
 143,
 143,
 142,
 142,
 141,
 141,
 141,
 140,
 140,
 139,
 139,
 139,
 138,
 137,
 136,
 136,
 135,
 135,
 134,
 132,
 132,
 131,
 131,
 129,
 129,
 129,
 127

In [57]:
def create_training_example_compact(solver_def, in_grid, out_grid):
    prompt = solver_def.split("\n", 1)
    prompt = [prompt[0], f'''    """
    Example input:
    {"\n    ".join([row for row in grid_to_string_compact(in_grid).split("\n")])}
    Example output:
    {"\n    ".join([row for row in grid_to_string_compact(out_grid).split("\n")])}
    """''', prompt[1]]
    return "\n".join(prompt)

training_examples = []
for label in train_task_labels:
    solver_def = inspect.getsource(getattr(solvers, f'solve_{label}'))
    N = len(train_tasks[label]['train'])
    M = len(train_tasks[label]['test'])
    for i in range(N):
        in_grid = train_tasks[label]['train'][i]['input']
        out_grid = train_tasks[label]['train'][i]['output']
        training_examples.append(create_training_example_compact(solver_def, in_grid, out_grid))
    for i in range(M):
        in_grid = train_tasks[label]['test'][i]['input']
        out_grid = train_sols[label][i]
        training_examples.append(create_training_example_compact(solver_def, in_grid, out_grid))

In [58]:
len(training_examples)

1718

In [62]:
training_example_lengths = [len(codegen_tokenizer(ex)['input_ids']) for ex in training_examples]

In [64]:
sorted(training_example_lengths, reverse = True)

[1470,
 1465,
 1464,
 1412,
 1394,
 1391,
 1347,
 1311,
 1254,
 1233,
 1209,
 1187,
 1171,
 1113,
 1091,
 1088,
 1083,
 1061,
 1044,
 1030,
 1030,
 1024,
 1019,
 1017,
 1012,
 1010,
 1000,
 997,
 993,
 979,
 978,
 965,
 965,
 960,
 959,
 947,
 929,
 925,
 919,
 917,
 911,
 908,
 902,
 891,
 872,
 871,
 861,
 861,
 860,
 860,
 851,
 848,
 840,
 837,
 833,
 829,
 824,
 819,
 818,
 813,
 808,
 803,
 796,
 794,
 794,
 793,
 792,
 779,
 773,
 772,
 771,
 768,
 764,
 758,
 755,
 751,
 750,
 749,
 745,
 741,
 739,
 737,
 736,
 722,
 720,
 718,
 717,
 705,
 704,
 703,
 699,
 699,
 692,
 689,
 686,
 685,
 685,
 681,
 680,
 680,
 674,
 669,
 669,
 667,
 666,
 666,
 660,
 660,
 660,
 660,
 659,
 652,
 650,
 650,
 649,
 649,
 644,
 644,
 643,
 643,
 642,
 638,
 637,
 637,
 637,
 636,
 635,
 635,
 635,
 635,
 634,
 632,
 632,
 631,
 630,
 629,
 628,
 626,
 625,
 625,
 622,
 621,
 619,
 619,
 619,
 618,
 615,
 615,
 615,
 612,
 611,
 610,
 609,
 609,
 609,
 606,
 605,
 598,
 597,
 597,
 596,
 596,
 

Worst case length 1470! Fits in 2048 context window.

In [65]:
np.argmax(training_example_lengths)

233

In [67]:
print(training_examples[233])

def solve_264363fd(I):
    """
    Example input:
    888888888888888888888888888888
    888888888888888888888888888888
    888888888883333333333333333888
    888888888883333333333333333888
    888888888883333333333333333888
    888888888883333333333333333888
    888888888883333333333334333888
    888888888883333333333333333888
    888888888883333333333333333888
    833333333383333333333333333888
    833333333383333433333333333888
    833333333383333333333333333888
    833343333383333333333333333888
    833333333383333333333333333888
    833333333383333333333333333888
    833333333388888888888888888888
    833333333388333333333388888888
    833333333388333333333388888888
    833333333388333333333388888888
    833333333388333333333388888888
    833333333388333333433388888888
    833333333388333333333388888888
    833333333388333333333388888888
    833333333388888888888888888888
    888888888888888888888888588888
    888888888888888888888886568888
    888888888888888888888886468888
    8

In [83]:
print(training_examples[23])

def solve_0520fde7(I):
    """
    Example input:
    1005010
    0105111
    1005000
    Example output:
    000
    020
    000
    """
    x1 = vmirror(I)
    x2 = lefthalf(x1)
    x3 = righthalf(x1)
    x4 = vmirror(x3)
    x5 = cellwise(x2, x4, ZERO)
    O = replace(x5, ONE, TWO)
    return O



In [108]:
input_string = '''def solve_0520fde7(I):
    """
    Example input:
    1005010
    0105111
    1005000
    Example output:
    000
    020
    000
    """
    x1 = vmirror(I)
    x2 = lefthalf(x1)
    x3 = righthalf(x1)
    '''

inputs = codegen_tokenizer(input_string, return_tensors = 'pt')
for key in inputs:
    inputs[key] = inputs[key].to(DEVICE)
outputs = codegen.generate(**inputs,
                         max_new_tokens = 100,
                         do_sample = True,
                         top_k = 10)
text = codegen_tokenizer.batch_decode(outputs)[0]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [109]:
print(text)

def solve_0520fde7(I):
    """
    Example input:
    1005010
    0105111
    1005000
    Example output:
    000
    020
    000
    """
    x1 = vmirror(I)
    x2 = lefthalf(x1)
    x3 = righthalf(x1)
    y1 = x1 + x2 + x3
    y2 = vmirror(y1)
    y3 = lefthalf(y2)
    y4 = righthalf(y2)
    y5 = vmirror(y3)
    y6 = lefthalf(y4)
    y7 = righthalf(y4)
    y8 = vmirror(y5)
    y


In [99]:
outputs

tensor([[ 4299,  8494,    62,  2713,  1238,    69,  2934,    22,     7,    40,
          2599,   198, 50284, 37811,   198, 50284, 16281,  5128,    25,   198,
         50284,  3064,    20, 20943,   198, 50284,   486,  2713, 16243,   198,
         50284,  3064, 27641,   198, 50284, 16281,  5072,    25,   198, 50284,
           830,   198, 50284, 33618,   198, 50284,   830,   198, 50284, 37811,
           198, 50284,    87,    16,   796,   410, 10793,  1472,     7,    40,
             8,   198, 50284,    87,    17,   796,   443,    69,   400,  1604,
             7,    87,    16,     8,   198, 50284,    87,    18,   796,   374,
         10887,  1604,     7,    87,    16,     8,   198, 50284,    87,    19,
           796,   374, 10887,  1604,     7,    87,    17,     8,   198, 50284,
            87,    20,   796,   374, 10887,  1604,     7,    87,    18,     8,
           198, 50284,    87,    21,   796,   374, 10887,  1604,     7,    87,
            19,     8,   198, 50284,    87,    22,  